<a href="https://colab.research.google.com/github/z0li627/DS-Unit-2-Kaggle-Challenge/blob/master/Zoltan_Gaspar_assignment_kaggle_challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

# Decision Trees

## Assignment
- [ ] [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website. You will be given the URL in Slack. Go to the Rules page. Accept the rules of the competition.
- [ ] Do train/validate/test split with the Tanzania Waterpumps data.
- [ ] Begin with baselines for classification.
- [ ] Select features. Use a scikit-learn pipeline to encode categoricals, impute missing values, and fit a decision tree classifier.
- [ ] Get your validation accuracy score.
- [ ] Get and plot your feature importances.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

### Reading

- A Visual Introduction to Machine Learning
  - [Part 1: A Decision Tree](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)
  - [Part 2: Bias and Variance](http://www.r2d3.us/visual-intro-to-machine-learning-part-2/)
- [Decision Trees: Advantages & Disadvantages](https://christophm.github.io/interpretable-ml-book/tree.html#advantages-2)
- [How a Russian mathematician constructed a decision tree — by hand — to solve a medical problem](http://fastml.com/how-a-russian-mathematician-constructed-a-decision-tree-by-hand-to-solve-a-medical-problem/)
- [How decision trees work](https://brohrer.github.io/how_decision_trees_work.html)
- [Let’s Write a Decision Tree Classifier from Scratch](https://www.youtube.com/watch?v=LDRbO9a6XPU) — _Don’t worry about understanding the code, just get introduced to the concepts. This 10 minute video has excellent diagrams and explanations._
- [Random Forests for Complete Beginners: The definitive guide to Random Forests and Decision Trees](https://victorzhou.com/blog/intro-to-random-forests/)


### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Define a function to wrangle train, validate, and test sets in the same way. Clean outliers and engineer features. (For example, [what columns have zeros and shouldn't?](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values) What columns are duplicates, or nearly duplicates? Can you extract the year from date_recorded? Can you engineer new features, such as the number of years from waterpump construction to waterpump inspection?)
- [ ] Try other [scikit-learn imputers](https://scikit-learn.org/stable/modules/impute.html).
- [ ] Make exploratory visualizations and share on Slack.


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

For this problem, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this classification problem, you may want to use the parameter `logistic=True`, but it can be slow.

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from a previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```


In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

     |████████████████████████████████| 102kB 2.5MB/s 


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
train, valid = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)


train.shape, test.shape, valid.shape

((47520, 41), (14358, 40), (11880, 41))

In [6]:
import plotly.express as px
px.scatter(train, x='longitude', y='latitude', color='status_group', opacity=0.1)

In [7]:
train.isnull().sum(), test.isnull().sum(), valid.isnull().sum()

(id                           0
 amount_tsh                   0
 date_recorded                0
 funder                    2904
 gps_height                   0
 installer                 2917
 longitude                    0
 latitude                     0
 wpt_name                     0
 num_private                  0
 basin                        0
 subvillage                 286
 region                       0
 region_code                  0
 district_code                0
 lga                          0
 ward                         0
 population                   0
 public_meeting            2644
 recorded_by                  0
 scheme_management         3128
 scheme_name              22532
 permit                    2443
 construction_year            0
 extraction_type              0
 extraction_type_group        0
 extraction_type_class        0
 management                   0
 management_group             0
 payment                      0
 payment_type                 0
 water_q

In [8]:
train['status_group'].value_counts(normalize=True)*100

functional                 54.307660
non functional             38.423822
functional needs repair     7.268519
Name: status_group, dtype: float64

In [11]:
valid['status_group'].value_counts(normalize=True)*100

functional                 54.309764
non functional             38.425926
functional needs repair     7.264310
Name: status_group, dtype: float64

In [0]:
import numpy as np

def cleanup(X):
    X = X.copy()
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    zcols = ['longitude', 'latitude']
    for col in zcols:
        X[col] = X[col].replace(0, np.nan)
            
    X = X.drop(columns='quantity_group')
    return X


train = cleanup(train)
valid = cleanup(valid)
test = cleanup(test)

In [14]:
train.shape, valid.shape, test.shape

((47520, 40), (11880, 40), (14358, 39))

In [16]:
target = 'status_group'
train_feat = train.drop(columns=[target, 'id'])
num_feat = train_feat.select_dtypes(include='number').columns.tolist()
cardi = train_feat.select_dtypes(exclude='number').nunique()
categ_feat = cardi[cardi <= 50].index.tolist() 
features = num_feat + categ_feat
print(features)

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year', 'basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


In [0]:
X_train = train[features]
y_train = train[target]
X_valid = valid[features]
y_valid = valid[target]
X_test = test[features]

In [31]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipel = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    LogisticRegression(multi_class='auto', solver='newton-cg', n_jobs=-1, max_iter=100)
)

pipel.fit(X_train, y_train)
print('Validation accuracy: ', pipel.score(X_valid, y_valid)*100, '%')
y_pred = pipel.predict(X_test)


Validation accuracy:  73.08922558922559 %


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

pipel2 = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    RandomForestClassifier(n_estimators=100, max_depth=27, random_state=0)
)

pipel2.fit(X_train, y_train)
print('Validation accuracy: ', pipel2.score(X_valid, y_valid)*100, '%')

Validation accuracy:  80.84175084175084 %


In [0]:
y_pred2 = pipel2.predict(X_test)
submi = sample_submission.copy()
submi['status_group'] = y_pred2
submi.to_csv('gz_predicts.csv', index=False)

In [46]:
import plotly.express as px
px.scatter(train, x='longitude', y='latitude', color='status_group', opacity=0.1)

In [48]:
px.scatter(valid, x='longitude', y='latitude', color='status_group', opacity=0.1)

In [0]:
test2 = test
test2['status_group'] = y_pred2

In [51]:
px.scatter(test2, x='longitude', y='latitude', color='status_group', opacity=0.1)